In [ ]:
# Installing and Importing required libs
!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install rouge_score
!pip install evaluate
import zipfile
import os
import glob
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import evaluate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
# Downloading the Dataset and Unzipping

!wget -O master.zip https://github.com/ryanzhumich/AESLC/archive/refs/heads/master.zip
%rm -rf /content/master/

def extract_zip(zip_path, extract_to):
    # Ensure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    # Open the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents to the specified directory
        zip_ref.extractall(extract_to)

zip_path = '/content/master.zip'
extract_to = '/content/master'
extract_zip(zip_path, extract_to)

--2024-07-22 07:55:00--  https://github.com/ryanzhumich/AESLC/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ryanzhumich/AESLC/zip/refs/heads/master [following]
--2024-07-22 07:55:01--  https://codeload.github.com/ryanzhumich/AESLC/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [     <=>            ]  11.11M  12.5MB/s    in 0.9s    

2024-07-22 07:55:02 (12.5 MB/s) - ‘master.zip’ saved [11650497]



In [ ]:
# Processing the downloaded datasets and populating train, dev and test sets.

train_directory_path  = '/content/master/AESLC-master/enron_subject_line/train'
dev_directory_path  = '/content/master/AESLC-master/enron_subject_line/dev'
test_directory_path  = '/content/master/AESLC-master/enron_subject_line/test'

def load_files(directory, has_annotations=False):
    files = glob.glob(os.path.join(directory, "*"))
    data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as file:
            content = file.read()
        email_body = ""
        subject_line = ""
        email_body, subjects = content.split('@subject\n', 1)
        annotations = []
        if has_annotations:
          subjects = subjects.replace("@ann0", "@subject\n").replace("@ann1", "@subject\n").replace("@ann2", "@subject\n")
          subject_line, annotation_1, annotation_2, annotation_3 = subjects.split('@subject\n')
          subject_line = subject_line.strip()
          annotations = [annotation_1.strip(), annotation_2.strip(), annotation_3.strip()]
          data.append((email_body, subject_line, annotations))
        else:
          subject_line = subjects.strip()
          data.append((email_body, subject_line))
    return data

# Load the data
dev_data = load_files(dev_directory_path, has_annotations=True)
train_data = load_files(train_directory_path)
test_data = load_files(test_directory_path, has_annotations=True)

# Check dataset lengths
print(f"Train dataset length: {len(train_data)}")
print(f"Dev dataset length: {len(dev_data)}")
print(f"Test dataset length: {len(test_data)}")

Train dataset length: 14436
Dev dataset length: 1960
Test dataset length: 1906


In [ ]:
# Improrting the model form Hugging face

# Load the tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# No of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

124,439,808 total parameters.
124,439,808 training parameters.


In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id, tokenizer.eos_token, tokenizer.eos_token_id

(None, None, '<|endoftext|>', 50256)

In [ ]:
#  ZERO SHOT TESTING with random prompt

#Define your input text

input_text = "Who is the president of USA?"
input_ids = tokenizer.encode(input_text,
                             max_length=512,
                             truncation=True,
                             return_tensors='pt',
                             ).to(device)

# Create an attention mask and move it to the same device
attention_mask = torch.ones(input_ids.shape, device=device)

# Check the shapes of input tensors
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention mask shape: {attention_mask.shape}")

# Generate text
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens = 12,
    top_p=0.9,
    top_k=50
)

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Input IDs shape: torch.Size([1, 7])
Attention mask shape: torch.Size([1, 7])


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Who is the president of USA?

The president of USA is the president of the United


In [ ]:
# Zero Shot testing with email sample

def summarize_email(email_text, model, tokenizer, max_new_tokens=12):

    prompt = f"Summarize the following text: {email_text}"
    # Encode the prompt and move tensors to the same device as the model
    input_ids = tokenizer.encode(prompt,
                             max_length=512,
                             truncation=True,
                             return_tensors='pt',
                             ).to(device)

    # Generate text
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask = torch.ones(input_ids.shape, device=device),
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = generated_text.replace(prompt, "").strip()

    return generated_text

# Test summarization
email_body = train_data[0][0]
subject_line = train_data[0][1]
print("Email Body:", email_body)
print("Actual Subject Line:", subject_line)

summary = summarize_email(email_body, model, tokenizer)
print("\nGenerated Summary:\n", summary)

Email Body: Just a note to let you know that per my conversation with Tanya, after review  by Brent, today I will be sending out am amendment to BMO allowing for the  provision of Cash as Eligible Collateral under the ISDA.
When we get this in  signed I will send my usual email out to the world explaining this amendment,  but you may want to think about what procedures you will need to put in place  to make sure this happens EVERY time we take collateral in (not to mention,  we will also have to pay Canadian counsel to file releases as well , and this  will cost us money too!
We don't want to have to do this too often.)
Anything you want to tell me before I send this out today?
!


Actual Subject Line: Bank of Montreal

Generated Summary:
 Summarize the following text: Just a note to let you know that per my conversation with Tanya, after review  by Brent, today I will be sending out am amendment to BMO allowing for the  provision of Cash as Eligible Collateral under the ISDA.
When we 

In [ ]:
# Create Tokenized data loaders for Train, Dev and Test sets

class EmailDataset(Dataset):

    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        email_body, subject_line = self.data[index][:2]
        prompt = f"Summarize the following text: {email_body}"

        # Encode the entire prompt and target
        input_ids = self.tokenizer.encode(prompt,
                                max_length=self.max_length,
                                truncation=True,
                                return_tensors='pt',
                            )
        if input_ids.shape[1] < self.max_length:
          pad_i = torch.tensor([tokenizer.eos_token_id] * (self.max_length - input_ids.shape[1])).unsqueeze(0)
          input_ids = torch.cat((input_ids , pad_i), dim=1)

        target_encoding = self.tokenizer.encode(
                                subject_line,
                                truncation=True,
                                max_length=self.max_length,
                                return_tensors="pt"
                            )
        if target_encoding.shape[1] < self.max_length:
          pad_t = torch.tensor([tokenizer.eos_token_id] * (self.max_length - target_encoding.shape[1])).unsqueeze(0)
          target_encoding = torch.cat((target_encoding , pad_t), dim=1)

        return {
            'input_ids': input_ids,
            'attention_mask': torch.ones(input_ids.shape),
            'labels': target_encoding
        }

train_data_tokenized = EmailDataset(train_data, tokenizer)
dev_data_tokenized = EmailDataset(dev_data, tokenizer)
test_data_tokenized = EmailDataset(test_data, tokenizer)

In [ ]:
# Zero Shot testing with tokenized data loader
for email in train_data_tokenized:
  prmt = email['input_ids'].clone()
  attn = email['attention_mask'].clone()
  outputs = model.generate(
      input_ids=prmt.to(device),
      attention_mask = attn.to(device),
      max_new_tokens=12,
      pad_token_id=tokenizer.eos_token_id,
  )

  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_text = generated_text.replace(tokenizer.decode(prmt.squeeze()), "").strip()
  print(generated_text)
  break

Summarize the following text: Just a note to let you know that per my conversation with Tanya, after review  by Brent, today I will be sending out am amendment to BMO allowing for the  provision of Cash as Eligible Collateral under the ISDA.
When we get this in  signed I will send my usual email out to the world explaining this amendment,  but you may want to think about what procedures you will need to put in place  to make sure this happens EVERY time we take collateral in (not to mention,  we will also have to pay Canadian counsel to file releases as well, and this  will cost us money too!
We don't want to have to do this too often.)
Anything you want to tell me before I send this out today?
!

The U.S. Supreme Court has ruled that the U


In [ ]:
# Define Trainer and Compute Metrics

# Load ROUGE metric

rouge_metric = evaluate.load("rouge")

tokenizer.pad_token = tokenizer.eos_token

# Compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
    decoded_preds = tokenizer.batch_decode(predictions.squeeze(), skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels.squeeze(), skip_special_tokens=True)

    # Calculate ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=['rouge1', 'rouge2', 'rougeL'])
    return {k: round(v, 4) for k, v in result.items()}

def preprocess_logits_for_metrics(logits, labels):
    """
    Convert logits to token IDs for metric computation.

    Args:
        logits (torch.Tensor): The raw model outputs (logits) with shape [batch_size, sequence_length, vocab_size].
        labels (torch.Tensor): The ground truth labels with shape [batch_size, sequence_length].

    Returns:
        tuple: Processed token IDs for predictions and labels.
    """
    # Convert logits to predicted token IDs
    # logits shape: (batch_size, sequence_length, vocab_size)
    pred_ids = torch.argmax(logits, dim=-1)  # shape: (batch_size, sequence_length)

    # Return the token IDs as-is along with labels
    return pred_ids, labels

In [ ]:
# GPT2 Model Training

# Define training arguments
training_args = TrainingArguments(
    output_dir='results_gpt2_small',  # output directory
    num_train_epochs=5,               # number of epochs
    eval_strategy="epoch",            # evaluate every epoch
    learning_rate=2e-4,               # learning rate
    per_device_train_batch_size=4,    # batch size for training
    per_device_eval_batch_size=4,     # batch size for evaluation
    save_strategy='epoch',            # save checkpoints at end of epoch
    save_total_limit=1,               # save only latest check point
    dataloader_num_workers=2,         # number of worker nodes
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=dev_data_tokenized,
    data_collator = data_collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
train_result = trainer.train()

# Save the model
trainer.save_model()

# Evaluate the model
eval_result = trainer.evaluate()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,2.984600,3.006810,0.316900,0.104600,0.237400
2,2.608300,2.979860,0.435700,0.153900,0.337000
3,2.280900,3.027455,0.363400,0.129700,0.279700


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,2.984600,3.006810,0.316900,0.104600,0.237400
2,2.608300,2.979860,0.435700,0.153900,0.337000
3,2.280900,3.027455,0.363400,0.129700,0.279700
4,1.992400,3.107131,0.447400,0.162700,0.346500
5,1.786700,3.195985,0.464600,0.167800,0.360400


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
tokenizer.save_pretrained('results_gpt2_small')
!zip -r {'results_gpt2_small'} {'results_gpt2_small'}

  adding: results_gpt2_small/ (stored 0%)
  adding: results_gpt2_small/tokenizer_config.json (deflated 55%)
  adding: results_gpt2_small/generation_config.json (deflated 24%)
  adding: results_gpt2_small/merges.txt (deflated 53%)
  adding: results_gpt2_small/training_args.bin (deflated 51%)
  adding: results_gpt2_small/model.safetensors (deflated 7%)
  adding: results_gpt2_small/checkpoint-18045/ (stored 0%)
  adding: results_gpt2_small/checkpoint-18045/generation_config.json (deflated 24%)
  adding: results_gpt2_small/checkpoint-18045/rng_state.pth (deflated 25%)
  adding: results_gpt2_small/checkpoint-18045/trainer_state.json (deflated 74%)
  adding: results_gpt2_small/checkpoint-18045/training_args.bin (deflated 51%)
  adding: results_gpt2_small/checkpoint-18045/model.safetensors (deflated 7%)
  adding: results_gpt2_small/checkpoint-18045/config.json (deflated 51%)
  adding: results_gpt2_small/checkpoint-18045/optimizer.pt (deflated 8%)
  adding: results_gpt2_small/checkpoint-18045/

In [ ]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/content/results_gpt2_small/checkpoint-18045")
tokenizer = GPT2Tokenizer.from_pretrained("/content/results_gpt2_small")

In [ ]:
import huggingface_hub

model_name = "gpt2_for_email_summarization_enron"  # Choose a name for your model
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/paramasivan27/gpt2_for_email_summarization_enron/commit/c9fdc4cdb71432e2ca6a7fa8bc1da8dcfa6e257d', commit_message='Upload tokenizer', commit_description='', oid='c9fdc4cdb71432e2ca6a7fa8bc1da8dcfa6e257d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# verify on test data
model.to(device)

for i in range(5):
  prompt = f"Summarize the following text: {test_data[i][0]}"

  input_ids = tokenizer.encode(prompt, return_tensors='pt', max_length=1024, truncation=True).to(device)

  # Generate text
  outputs = model.generate(
  input_ids=input_ids,
  attention_mask = torch.ones(input_ids.shape, device=device).to(device),
  max_new_tokens=12,
  pad_token_id=tokenizer.eos_token_id,
  )

  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_text = generated_text.replace(prompt, "").strip()
  print(f"email_body : {test_data[i][0]}")
  print(f"Actual Subject : {test_data[i][1]}")
  print(f"Generated Subject After training : {generated_text}")
  print('--------------------------------------------------------------------------------------------------------')

email_body : John,   I am working with Gerald and Debra on implementing a GISB with Tenn. Gas.
Currently, we have worked out every detail.
Their legal department is reviewing our special provisions and then they are supposed to sign two copies of the agreement and send them to us for final execution.
If you have any questions with respect to this Agreement, please do not hesitate to contact me.
Best Regards,


Actual Subject : Tennessee Gas Pipeling GISB
Generated Subject After training : marcello

marci x57566

marci
--------------------------------------------------------------------------------------------------------
email_body : At Richard Sanders' request, we ask that you freeze destruction of electronic mail for the following individuals (effective immediately).
This request is made in regard to the Bid/Ask patent litigation.
We should retain their e-mail communications going back as far as possible.
Jay Webb 	Kevin Montagne 	Jarod M. Jenson 	Mark E. Taylor (legal) 	Michael Guad